In [1]:
import json

with open("./params.json", mode = "r", encoding = "utf-8") as f:
    data = json.load(f)
    seed_val = data["seed_val"]
    model_path = data["model_path"]
    dataset_path_train = data["dataset_path"]["train"]
    dataset_path_val = data["dataset_path"]["validation"]
    dataset_path_test = data["dataset_path"]["test"]
    num_single_sample_timesteps = data["num_single_sample_timesteps"]
    window_stride = data["window_stride"]
    input_window_length = data["input_window_length"]
    label_window_length = data["label_window_length"]
    input_features = data["input_features"]
    label_features = data["label_features"]
    relative_attention_num_buckets = data["relative_attention_num_buckets"]
    embedding_dim = data["embedding_dim"]
    num_attention_head = data["num_attention_head"]
    num_encoder_layers = data["num_encoder_layers"]
    num_decoder_layers = data["num_decoder_layers"]
    position_wise_nn_dim = data["position_wise_nn_dim"]
    dropout = data["dropout"]
    batch_size = data["batch_size"]
    epochs = data["epochs"]
    learning_rate = data["learning_rate"]

In [2]:
import random
import numpy as np
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(seed_val)
random.seed(seed_val)
np.random.seed(seed_val)

In [3]:
from torch.utils.data import DataLoader
from torchmetrics.regression import R2Score
from datetime import datetime

from utils.pipeline.Data import get_mean_std_respected_temporal, WindowedIterableDataset
from utils.pipeline.Model import TimeSeriesHuggingFaceTransformer
from utils.pipeline.Run import train, validate
from utils.pipeline.Monitor import Overfit

/users/labnet5/gr5/abahari/Documents/Thesis/src/thesis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
##### TRAIN #####
# stats = get_mean_std(
#     dataset_path = dataset_path_train,
#     cols = input_features
# )

stats = get_mean_std_respected_temporal(
    dataset_path = dataset_path_train,
    cols = input_features,
    num_single_sample_timesteps = num_single_sample_timesteps,
    input_window_len = input_window_length,
    label_window_len = label_window_length,
    window_stride = window_stride
)

df_train = WindowedIterableDataset(
    dataset_path = dataset_path_train,
    stats = stats,
    input_features = input_features,
    label_features = label_features,
    num_single_sample_timesteps = num_single_sample_timesteps,
    stride = window_stride,
    input_window_length = input_window_length,
    label_window_length = label_window_length
)

data_loader_train = DataLoader(
    df_train,
    batch_size = batch_size,
    # num_workers = 0,
    # prefetch_factor = 12,
    # persistent_workers = False,
    pin_memory = True
)



##### VALIDATION #####
df_val = WindowedIterableDataset(
    dataset_path = dataset_path_val,
    stats = stats,
    input_features = input_features,
    label_features = label_features,
    num_single_sample_timesteps = num_single_sample_timesteps,
    stride = window_stride,
    input_window_length = input_window_length,
    label_window_length = label_window_length
)

data_loader_val = DataLoader(
    df_val,
    batch_size = batch_size,
    # num_workers = 0,
    # prefetch_factor = 12,
    # persistent_workers = False,
    pin_memory = True
)



##### TEST #####
df_test = WindowedIterableDataset(
    dataset_path = dataset_path_test,
    stats = stats,
    input_features = input_features,
    label_features = label_features,
    num_single_sample_timesteps = num_single_sample_timesteps,
    stride = window_stride,
    input_window_length = input_window_length,
    label_window_length = label_window_length
)

data_loader_test = DataLoader(
    df_test,
    batch_size = batch_size,
    # num_workers = 0,
    # prefetch_factor = 12,
    # persistent_workers = False,
    pin_memory = True
)

In [5]:
model = TimeSeriesHuggingFaceTransformer(
    input_window_len = input_window_length,
    output_window_len = label_window_length,
    input_dim = len(input_features),
    output_dim = len(label_features),
    d_model = embedding_dim,
    num_head = num_attention_head,
    num_encoder_layers = num_encoder_layers,
    num_decoder_layers = num_encoder_layers,
    position_wise_ffn_dim = position_wise_nn_dim,
    relative_attention_num_buckets = relative_attention_num_buckets,
    dropout = dropout
).to(device)

overfit_monitor = Overfit()

print(f"Number of trainable parameters in the model: {sum(p.numel() for p in model.parameters() if p.requires_grad)}\n")

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(),
    lr = learning_rate
)

train_r2 = R2Score(multioutput = "uniform_average").to(device)
val_r2 = R2Score(multioutput = "uniform_average").to(device)

train_per_timestep_r2 = [R2Score(multioutput = "uniform_average").to(device) for _ in range(label_window_length)]
val_per_timestep_r2 = [R2Score(multioutput = "uniform_average").to(device) for _ in range(label_window_length)]

overfit_count = 0

for epoch in range(epochs):
    train_loss, train_r2_value = train(
        model = model,
        optimizer = optimizer,
        criterion = criterion,
        r2 = train_r2,
        per_timestep_r2 = train_per_timestep_r2,
        data_loader = data_loader_train,
        device = device,
        epoch = epoch,
        total_epochs = epochs
    )

    val_loss, val_r2_value = validate(
        model = model,
        criterion = criterion,
        r2 = val_r2,
        per_timestep_r2 = val_per_timestep_r2,
        data_loader = data_loader_val,
        device = device,
        epoch = epoch,
        total_epochs = epochs
    )

    # if(overfit_monitor.check(epoch = epoch, train_loss = train_loss, val_loss = val_loss)):
    #     break

Number of trainable parameters in the model: 1170821



Epoch 1/25: 717it [00:58, 12.27it/s, train_loss=0.027629]


Epoch [1/25], Train Loss: 0.093862, Train R2: 0.925757

Worst 5 Time-Steps Train R2:
    Time-step 1: R2 = 0.765699
    Time-step 2: R2 = 0.903209
    Time-step 3: R2 = 0.914805
    Time-step 4: R2 = 0.921560
    Time-step 50: R2 = 0.923938
Best 5 Time-Steps Train R2:
    Time-step 12: R2 = 0.932903
    Time-step 11: R2 = 0.932694
    Time-step 13: R2 = 0.932450
    Time-step 19: R2 = 0.932355
    Time-step 23: R2 = 0.932207



Epoch 1/25: 207it [00:31,  6.51it/s, val_loss=0.485984]


Epoch [1/25], Val Loss: 0.713631, Val R2: 0.447250

Worst 5 Time-Steps Val R2:
    Time-step 50: R2 = -0.036928
    Time-step 49: R2 = -0.022823
    Time-step 48: R2 = -0.012121
    Time-step 47: R2 = -0.001138
    Time-step 46: R2 = 0.012309
Best 5 Time-Steps Val R2:
    Time-step 1: R2 = 0.971244
    Time-step 2: R2 = 0.957100
    Time-step 3: R2 = 0.941425
    Time-step 4: R2 = 0.926338
    Time-step 5: R2 = 0.910963

-----------------------------------------------------------------



...

Epoch [24/25], Val Loss: 0.458097, Val R2: 0.646090

Worst 5 Time-Steps Val R2:
    Time-step 50: R2 = 0.270183
    Time-step 49: R2 = 0.283508
    Time-step 48: R2 = 0.295797
    Time-step 47: R2 = 0.309071
    Time-step 46: R2 = 0.323396
Best 5 Time-Steps Val R2:
    Time-step 1: R2 = 0.988960
    Time-step 2: R2 = 0.979865
    Time-step 3: R2 = 0.970929
    Time-step 4: R2 = 0.961288
    Time-step 5: R2 = 0.951546

-----------------------------------------------------------------



Epoch 25/25: 717it [00:55, 12.91it/s, train_loss=0.013757]


Epoch [25/25], Train Loss: 0.020191, Train R2: 0.983735

Worst 5 Time-Steps Train R2:
    Time-step 1: R2 = 0.974932
    Time-step 2: R2 = 0.981290
    Time-step 3: R2 = 0.982592
    Time-step 4: R2 = 0.983128
    Time-step 5: R2 = 0.983375
Best 5 Time-Steps Train R2:
    Time-step 33: R2 = 0.984194
    Time-step 24: R2 = 0.984189
    Time-step 23: R2 = 0.984161
    Time-step 21: R2 = 0.984148
    Time-step 17: R2 = 0.984143



Epoch 25/25: 207it [00:31,  6.49it/s, val_loss=0.270159]

Epoch [25/25], Val Loss: 0.465162, Val R2: 0.641302

Worst 5 Time-Steps Val R2:
    Time-step 50: R2 = 0.271085
    Time-step 49: R2 = 0.283702
    Time-step 48: R2 = 0.295199
    Time-step 47: R2 = 0.307392
    Time-step 46: R2 = 0.320595
Best 5 Time-Steps Val R2:
    Time-step 1: R2 = 0.988081
    Time-step 2: R2 = 0.979447
    Time-step 3: R2 = 0.970064
    Time-step 4: R2 = 0.960049
    Time-step 5: R2 = 0.950108

-----------------------------------------------------------------



In [6]:
torch.save(model, model_path)

description = '''
dataset: random
bos_projector: non-linear (1 LeakyReLU)
bos_input: encoder hidden state of last input time-step
positional encoding: sin, cos
seed_val: 7
num_single_sample_timesteps: 1000
input_window_len: 100
label_window_len: 50
window_stride: 10
relative_attention_num_buckets: 64
embedding_dim: 128
num_attention_head: 8
num_encoder_layers: 5
num_decoder_layers: 5
position_wise_nn_dim: 64
dropout: 0.5
batch_size: 128
epochs: 25
learning_rate: 0.0005
psi_e, b_e, psi_plus, b_plus, u_list -> psi_e, b_e, psi_plus, b_plus, u_list

'''

with open("./results/experiments.txt", mode = "a") as f:
    f.write(str(datetime.now()).replace(" ", "-") + "\n")
    f.write(f"{model_path}\n\n")
    f.write(f"Training Loss: {train_loss}, Training R2: {train_r2_value}\n")
    f.write(f"Validation Loss: {val_loss}, Validation R2: {val_r2_value}\n")
    f.write(description)
    f.write("==========================================\n")